In [1]:
import geopandas as gpd
import requests
import os
import zipfile
import random

### Get subtiles for area of choice

In [3]:

# Specify the URL of the GeoPackage file
admin_url = "https://geodata.ucdavis.edu/gadm/gadm4.1/gpkg/gadm41_NLD.gpkg"

# Download and load the GeoPackage
admin_areas = gpd.read_file(admin_url, layer='ADM_ADM_2')

target_area = 'Amsterdam'  # Change this to the desired polygon ID

selected_area = admin_areas[admin_areas['NAME_2'] == target_area]

# PART 2:
# Specify the URL of the zipped shapefile
shapefile_zip_url = "https://static.fwrite.org/2023/01/AHN_subunits_GeoTiles.zip"

# Define a temporary directory to store the downloaded file
temp_dir = os.path.join(os.getcwd(), 'temp')

# Create temp directory if it doesn't exist
os.makedirs(temp_dir, exist_ok=True)

# Download the zipped shapefile
shapefile_zip_path = os.path.join(temp_dir, "AHN_subunits_GeoTiles.zip")
response = requests.get(shapefile_zip_url)
with open(shapefile_zip_path, 'wb') as f:
    f.write(response.content)

# Unzip the shapefile
with zipfile.ZipFile(shapefile_zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# List the shapefile files in the unzipped directory
geotiles_files = [os.path.join(temp_dir, f) for f in os.listdir(temp_dir) if f.endswith('.shp')]

# Load the shapefile
geotiles = gpd.read_file(geotiles_files[0])

# Assuming you want to reproject geotiles to match the CRS of selected_area
# geotiles = geotiles.to_crs(selected_area.crs)

# Transform the selected_area to match the CRS of geotiles
selected_area = selected_area.to_crs(geotiles.crs)

# Identify which tiles are in the area
contained_polygons = gpd.overlay(geotiles, selected_area, how='intersection')

# Extract identifiers or any other attributes you need
tile_list = contained_polygons['GT_AHN'] 

subtile_list = contained_polygons['GT_AHNSUB'] 

# Testing purposes
sample_tile = sample(tile_list, 1)
sample_subtile = sample(subtile_list,1)

print(tile_list)
print(subtile_list.unique)


0      25EZ1_01
1      25EZ1_02
2      25EZ1_03
3      25EZ1_06
4      25EZ1_07
         ...   
204    25DN2_21
205    25DN2_22
206    25DN2_23
207    25DN2_24
208    25DN2_25
Name: GT_AHNSUB, Length: 209, dtype: object
0      25EZ1
1      25EZ1
2      25EZ1
3      25EZ1
4      25EZ1
       ...  
204    25DN2
205    25DN2
206    25DN2
207    25DN2
208    25DN2
Name: GT_AHN, Length: 209, dtype: object


### Download AHN DTM and DSM

In [ ]:
def download_and_process_dsm_dtm(tile_list):
    dsm_url = "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_tile_name.zip"
    dtm_url = "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_tile_name.zip"

    for tile_name in tile_list:
        # Construct the complete URL for the current tile
        dsm_tile_url = dsm_url.format(tile_name=tile_name)
        dtm_tile_url = dtm_url.format(tile_name=tile_name)

        # Check if the file already exists in the results folder
        dsm_save_path = os.path.join(dsm_save_directory, tile_name,".tiff")
        dtm_save_path = os.path.join(dtm_save_directory, tile_name,".tiff")
        
        if os.path.exists(dsm_save_path):
            print("Skipping", tile_name, "as the file already exists.")
            continue  # Skip to the next iteration of the loop
        
        print("Downloading", tile_name)
        # Download the file
        response = requests.get(dsm_tile_url, stream=True)
        if response.status_code == 200:
            # Write the file to disk
            with open(dsm_save_path, 'wb') as f:
                for chunk in tqdm(response.iter_content(chunk_size=8192), desc="Progress", unit="KB"):
                    if chunk:
                        f.write(chunk)
        else:
            print("Failed to download", tile_name)
        
        
        if response.status_code == 200:
            # Write the file to disk
            with open(dsm_save_path, 'wb') as f:
                for chunk in tqdm(response.iter_content(chunk_size=8192), desc="Progress", unit="KB"):
                    if chunk:
                        f.write(chunk)
                        
            # Optionally, save the CHM file
            # with open(chm_file_path, 'wb') as f:
            #     f.write(response.content)
            
        
    response = requests.get(shapefile_zip_url)
    with open(shapefile_zip_path, 'wb') as f:
    f.write(response.content)
    
    dsm_save_directory = "results/tile_name_dsm.tif"
    dtm_save_directory = "results/tile_name_dtm.tif"